In [81]:
import pandas as pd
import cv2
from PIL import Image, ImageDraw
import os

def process_image(file_name, image_size, glasses_color, line_width):
    if glasses_color == "cian":
        glasses_color = "cyan"
    try:
        # Load Haar Cascade classifiers
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

        # Load the image
        img = cv2.imread(file_name)
        if img is None:
            print(f"Error: Could not find or open file {file_name}.")
            return

        # Detect faces
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=6, minSize=(50, 50))

        # Check if any faces were found
        if len(faces) == 0:
            print("No faces detected.")
            return

        # Process the first face
        x, y, w, h = faces[0]
        face_img = img[y:y+h, x:x+w]
        roi_gray = gray[y:y+h, x:x+w]

        eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=10, minSize=(30, 30))

        # Resize the face image
        resized_face = cv2.resize(face_img, image_size)
        scale_x = image_size[0] / w
        scale_y = image_size[1] / h

        # Add glasses
        pil_img = Image.fromarray(cv2.cvtColor(resized_face, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(pil_img)

        if len(eyes) >= 2:
            centers = []
            for (ex, ey, ew, eh) in eyes[-2:]:
                ex = int(ex * scale_x)
                ey = int(ey * scale_y)
                ew = int(ew * scale_x)
                eh = int(eh * scale_y)
                
                center = (ex + ew // 2, ey + eh // 2)
                centers.append(center)
            
            # Sort centers by x-coordinate (leftmost first)
            centers = sorted(centers, key=lambda c: c[0])

            # Draw glasses
            radius = max(ew, eh) // 2 + 10
            for center in centers:
                draw.ellipse([center[0] - radius, center[1] - radius,
                              center[0] + radius, center[1] + radius],
                             outline=glasses_color, width=line_width)
            
            # Draw glasses bridge
            draw.line([(centers[0][0] + radius, centers[0][1]),
                       (centers[1][0] - radius, centers[1][1])], 
                      fill=glasses_color, width=line_width)
            
            side_length = radius // 2 + 10
            # Draw left glasses side
            draw.line([(centers[0][0] - radius - side_length, centers[0][1] + 3),
                       (centers[0][0] - radius, centers[0][1])],
                      fill=glasses_color, width=line_width)
            # Draw right glasses side
            draw.line([(centers[1][0] + radius, centers[1][1]),
                       (centers[1][0] + radius + side_length, centers[1][1] + 3)],
                      fill=glasses_color, width=line_width)
        else:
            print("Eyes not found or detected incorrectly.")

        # Save the processed image
        output_dir = "processed_imgs"
        os.makedirs(output_dir, exist_ok=True)
        base_file_name = os.path.basename(file_name)
        output_path = os.path.join(output_dir, f"processed_{base_file_name}")
        pil_img.save(output_path)
        print(f"Saved: {output_path}")
    except Exception as e:
        print(f"Error: {e}")

def main():
    # Load data
    df = pd.read_excel('lab6.xlsx')
    
    # Define variant
    variant = ord('A') % 5 + 1
    print(f"Variant: {variant}")
    row = df.iloc[variant - 1]
        
    file_name = "imgs/" + row['file name']
    image_size = tuple(map(int, row['image size'].split('x')))
    glasses_color = row['glasses color']
    line_width = int(row['line width'])

    print(f"Inputs: {file_name}, {image_size}, {glasses_color}, {line_width}")
    process_image(file_name, image_size, glasses_color, line_width)

main()


Variant: 1
Inputs: imgs/emma-watson2.jpg, (300, 300), red, 2
Saved: processed_imgs\processed_emma-watson2.jpg
